In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC, SVC, OneClassSVM
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer, FunctionTransformer, PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score,confusion_matrix

In [2]:
df = pd.read_csv('mirai3.csv', header=None)
labels = [1] * 71000 + [-1] * 29000
df['label'] = labels

X = df.drop('label', axis = 1)
y = df['label']

df

,0,1,2,3,4,5,6,7,8,9,...,106,107,108,109,110,111,112,113,114,label
0,1.89742,68.1526,46.927100,2.30082,68.6038,46.126200,4.74452,71.1236,38.470500,36.0737,...,-9.074680e-29,-2.880320e-17,143.15200,74.0,0.000003,104.652,6.621220e-12,2.175260e-27,6.392140e-16,1
1,1.75007,69.4285,118.530000,3.06442,74.7885,106.647000,11.08620,78.8105,59.996500,110.2570,...,0.000000e+00,0.000000e+00,1.00000,60.0,0.000000,60.000,0.000000e+00,0.000000e+00,0.000000e+00,1
2,2.74967,65.9995,95.995400,4.06400,71.1496,120.978000,12.08570,77.2540,81.886900,111.2570,...,0.000000e+00,0.000000e+00,1.00000,60.0,0.000000,60.000,0.000000e+00,0.000000e+00,0.000000e+00,1
3,1.20593,89.8507,80.829300,1.81715,95.6911,137.610000,6.57048,100.4730,159.338000,78.1247,...,0.000000e+00,0.000000e+00,1.00000,86.0,0.000000,86.000,0.000000e+00,0.000000e+00,0.000000e+00,1
4,1.02825,86.0000,0.000003,1.13366,86.0004,0.000808,2.07755,86.0922,0.175812,12.4676,...,0.000000e+00,0.000000e+00,4.93732,86.0,0.000002,86.000,2.728480e-12,0.000000e+00,0.000000e+00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.00603,60.1305,2.854850,1.11986,62.2229,43.963200,5.61876,73.8553,112.847000,107.6470,...,0.000000e+00,0.000000e+00,1.00000,60.0,0.000000,60.000,0.000000e+00,0.000000e+00,0.000000e+00,-1
99996,2.00582,60.0655,1.435830,2.11972,61.1742,24.454500,6.61853,71.7619,120.420000,108.6470,...,0.000000e+00,0.000000e+00,1.00000,60.0,0.000000,60.000,0.000000e+00,0.000000e+00,0.000000e+00,-1
99997,1.40708,92.7404,133.860000,1.90395,95.1054,213.901000,6.04193,95.6510,415.480000,72.9923,...,0.000000e+00,0.000000e+00,1.00000,86.0,0.000000,86.000,0.000000e+00,0.000000e+00,0.000000e+00,-1
99998,2.40571,89.9386,89.252800,2.90284,91.9687,158.937000,7.04075,94.2802,367.820000,73.9909,...,0.000000e+00,0.000000e+00,1.00000,86.0,0.000000,86.000,0.000000e+00,0.000000e+00,0.000000e+00,-1


first try after fine tune

In [3]:
import pickle
with open("X_train.pkl" , "rb") as f:
    X_train = pickle.load(f)
with open("y_train.pkl" , "rb") as f:
    y_train = pickle.load(f)
with open("X_test.pkl" , "rb") as f:
    X_test = pickle.load(f)
with open("y_test.pkl" , "rb") as f:
    y_test = pickle.load(f)

In [4]:
y_train.value_counts(),y_test.value_counts()

( 1    35500
 -1    14500
 Name: label, dtype: int64,
  1    35500
 -1    14500
 Name: label, dtype: int64)

From the training set select only the benign..
 

Form the OcSVM training set(benign only, 35500 samples)..


Sort them for convenience especially for the next part of Kitsune who treats the data as timeseries


In [5]:
train_df = X_train.copy()

train_df['label'] = y_train
X_train_one_class = train_df[train_df['label'] == 1 ]


X_train_OCSVM = X_train_one_class.drop('label', axis = 1)
X_train_OCSVM.sort_index()

,0,1,2,3,4,5,6,7,8,9,...,105,106,107,108,109,110,111,112,113,114
6,1.99938,114.0000,0.000109,1.99972,113.9980,0.11162,2.09528,111.9170,106.2300,5.11994,...,0.000000e+00,0.000000e+00,0.0,1.00000,114.0000,0.000000e+00,114.0000,0.000000e+00,0.000000e+00,0.000000e+00
7,1.86453,102.8030,182.506000,2.48821,103.0490,162.88300,7.14736,102.3650,159.0650,78.60640,...,0.000000e+00,0.000000e+00,0.0,1.00000,114.0000,0.000000e+00,161.2200,0.000000e+00,0.000000e+00,0.000000e+00
8,2.85173,106.7290,147.056000,3.47795,106.1980,140.61600,8.13752,103.7950,154.1090,79.59550,...,0.000000e+00,0.000000e+00,0.0,1.00000,114.0000,0.000000e+00,161.2200,0.000000e+00,0.000000e+00,0.000000e+00
12,2.89126,90.5670,590.223000,3.71838,93.7737,522.41100,8.49585,98.6401,335.1540,79.94450,...,9.094950e-13,-6.251840e-36,0.0,7.95863,60.0000,9.536740e-07,84.8528,1.286220e-12,2.484300e-29,2.731520e-17
20,1.32915,75.1596,36.389400,1.63052,77.7364,85.25630,3.16704,85.4821,220.7160,27.68580,...,2.254730e+02,0.000000e+00,0.0,2.93182,82.8717,1.551040e+01,133.7640,2.405720e+02,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70994,2.69409,77.8750,73.735000,4.04707,76.1190,94.79560,13.01620,77.1938,82.6367,110.52900,...,1.818990e-12,0.000000e+00,0.0,160.52900,82.0000,2.523190e-06,82.0000,6.366460e-12,0.000000e+00,0.000000e+00
70996,4.69405,79.6325,46.479700,6.04704,78.0641,71.09860,15.01620,77.8339,74.2972,112.52900,...,9.094950e-13,0.000000e+00,0.0,160.50100,82.0000,2.336020e-06,82.0000,5.456970e-12,0.000000e+00,0.000000e+00
70997,5.69403,80.0483,39.128300,7.04703,78.6226,62.89570,16.01620,78.0941,70.6743,113.52900,...,1.818990e-12,0.000000e+00,0.0,160.49300,82.0000,1.348700e-06,82.0000,1.818990e-12,0.000000e+00,0.000000e+00
70998,5.77598,76.5773,89.893500,7.34149,76.0860,95.13270,16.46280,76.9950,85.0603,114.13100,...,4.547470e-13,0.000000e+00,0.0,2.98982,60.0000,9.536740e-07,60.0000,9.094950e-13,0.000000e+00,0.000000e+00


In [6]:

pipe = Pipeline([('scaler', MinMaxScaler()), ('svc', OneClassSVM(nu=0.01, gamma=0.0001))])

pipe.fit(X_train_OCSVM)


Pipeline(steps=[('scaler', MinMaxScaler()),
                ('svc', OneClassSVM(gamma=0.0001, nu=0.01))])

predict on training set

In [9]:
preds = pipe.predict(X_train_OCSVM)
true = [1] * 35500

y_pred = pipe.predict(X_train)
cm = confusion_matrix(true, preds)

# Extract TP, TN, FP, and FN
TN, FP, FN, TP = cm.ravel()

TPR = TP / (TP + FN) if (TP + FN) > 0 else 0  # Sensitivity/Recall
FPR = FP / (FP + TN) if (FP + TN) > 0 else 0  # Probability of false alarm
FNR = FN / (TP + FN) if (TP + FN) > 0 else 0  # Miss Rate
TNR = TN / (TN + FP) if (TN + FP) > 0 else 0  # Specificity

print(f"True Positive Rate (TPR): {TPR}")
print(f"False Positive Rate (FPR): {FPR}")
print(f"False Negative Rate (FNR): {FNR}")
print(f"True Negative Rate (TNR): {TNR}")

True Positive Rate (TPR): 0.99
False Positive Rate (FPR): 0
False Negative Rate (FNR): 0.01
True Negative Rate (TNR): 0


predict on testing set

In [10]:
preds = pipe.predict(X_test)

accuracy_score(y_test, preds), recall_score(y_test, preds, average='weighted'), precision_score(y_test, preds, average='weighted'), f1_score(y_test, preds, average='weighted')

(0.93944, 0.93944, 0.9414612519583898, 0.9377416954904626)

0.9377 F1 Score for outlier detection method is not bad at all.. a little bit worse than Kitsune(0.9677)